# TouchFree Vending Machine

In [63]:
import copy
import time
import cv2
import numpy as np
from keras.models import load_model
import serial

# For Bluetooth Communication

In [64]:
portOk=False
startTime=time.time()
while(time.time()-startTime<=2):
    try:
        serialPort = serial.Serial(port = "COM8", baudrate=9600, #baudrate can be more.
                           bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE)
        time.sleep(0.1)
        serialPort.write(b"H")
        portOk=True
        break
    except:
        serialPort.close()
if(portOk==False):
    print("Port Busy!!")

1

# Onscreen Window Parameters

In [65]:
prediction = ''
action = ''
purchaseId = ''
waitTime = 0
timeLeft = 0
score = 0
img_counter = 500

gesture_names = {0: 'Fist',
                 1: 'L',
                 2: 'Okay',
                 3: 'Palm',
                 4: 'Peace'}

# Load Model

In [66]:
model = load_model('VGG_cross_validated.h5')

# Function to Label the Predicted Image among Predefined Classes

In [67]:
def predict_rgb_image(img):
    result = gesture_names[model.predict_classes(img)[0]]
    print(result)
    return (result)

# Function to Predict the Image

In [68]:
def predict_rgb_image_vgg(image):
    image = np.array(image, dtype='float32')
    image /= 255
    pred_array = model.predict(image)
    #print(f'pred_array: {pred_array}')
    result = gesture_names[np.argmax(pred_array)]
    #print(f'Result: {result}')
    #print(max(pred_array[0]))
    score = float("%0.2f" % (max(pred_array[0]) * 100))
    #print(result)
    return result, score

# Parameters to Apply for Different Filters and Setting thresholds   

In [69]:
# parameters
cap_region_x_begin = 0.5  # start point/total width
cap_region_y_end = 0.8  # start point/total width
threshold = 60  # binary threshold
blurValue = 41  # GaussianBlur parameter
bgSubThreshold = 50
learningRate = 0

#serial communication
trigger=False
resetTrigger=False

# variableslt
isBgCaptured = 0  # bool, whether the background captured
triggerSwitch = False  # if true, keyboard simulator works

# Utility Function to Create Background Model with The Provided Threshold 

In [70]:
def captureBackgnd(bgSubThreshold): #capture backgnd
    global bgModel
    global isBgCaptured
    bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
    time.sleep(2) 
    isBgCaptured = 1

# Function to Remove Background from Frame

In [71]:
def remove_background(frame):
    fgmask = bgModel.apply(frame, learningRate=learningRate)
    kernel = np.ones((3, 3), np.uint8)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    return res

# Function to Process Target Image by resizing, reshaping

In [72]:
def processTarget(thresh):
    target = np.stack((thresh,) * 3, axis=-1)
    target = cv2.resize(target, (224, 224))
    target = target.reshape(1, 224, 224, 3)
    return target

# Function to Get the binarized Image and showing different onscreen parameters on the Window

In [73]:
def getThresh(frame):
        img = remove_background(frame)
        img = img[0:int(cap_region_y_end * frame.shape[0]),
              int(cap_region_x_begin * frame.shape[1]):frame.shape[1]]  # clip the ROI
#         cv2.imshow('mask', img)

        # convert the image into binary image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
        # cv2.imshow('blur', blur)
        ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        cv2.putText(thresh, f"Prediction: {prediction} ({score}%)", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255, 100, 0))
        cv2.putText(thresh, f"Action: {action}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255, 100, 0))  # Draw the text
        cv2.putText(thresh, f"Wait For: {int(float(waitTime))}sec", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255, 100, 0))
        cv2.putText(thresh, f"Time Left: {int(float(timeLeft))}sec", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255, 100, 0))
        cv2.putText(thresh, f"Purchased Product Id: {purchaseId}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255, 100, 0))
        cv2.imshow('Prediction & Action', thresh)
#         cv2.imshow("Original", frame)
        return thresh

# To Get The Frame from Camera and Apply Smoothing Filter

In [74]:
def getFrame(camera):
    ret, frame = camera.read()
    frame = cv2.bilateralFilter(frame, 5, 50, 100)  # smoothing filter
    frame = cv2.flip(frame, 1)  # flip the frame horizontally
    cv2.rectangle(frame, (int(cap_region_x_begin * frame.shape[1]), 0),
                  (frame.shape[1], int(cap_region_y_end * frame.shape[0])), (255, 0, 0), 2)
    cv2.imshow("original",frame)
    return frame

# To Capture the Background for The First Time only

In [75]:
def triggerCapture():
    global startCapture
    if(startCapture == True):
        #resetWindowContent()
        captureBackgnd(bgSubThreshold)  #To Capture Only Once
#         print("Hello from capture")
        startCapture = False
        time.sleep(1)

# To Execute the Whole Process

In [76]:
def totalProcess(camera):
    frame = getFrame(camera)
    thresh = getThresh(frame)
    target = processTarget(thresh)
    return target

# Sets a Wait Timer 5secs to Allow User to Remove Hand

In [77]:
def OkayWait():
    global action
    global waitTime
    startTime = time.time()
    waitTime = time.time()-startTime
    while(waitTime<=5):
        waitTime = time.time()-startTime
        action = "Gesture(Okay) to confirm in 60"
        prediction, score = predict_rgb_image_vgg(totalProcess(camera))
        cv2.waitKey(10)
    waitTime = 0

# Sets a Countdown Timer 5secs to Allow User to Get Ready for the Next Gesture

In [78]:
def nextWait():
    global action
    global waitTime
    startTime = time.time()
    waitTime = time.time()-startTime
    while(waitTime<=5):
        waitTime = time.time()-startTime
        action = "Done!! Wait For Next."
        prediction, score = predict_rgb_image_vgg(totalProcess(camera))
        cv2.waitKey(10)

# Send Acknowledge and Read from com_port  

In [79]:
def serialProcess():
    global trigger
    global resetTrigger
    while(1):
        serialPort.write(b".")
        a=serialPort.read()
#         print(type(a))
#         print(a)
        if(a==b'O'):
            trigger = True
            break

# Start Camera

In [80]:
# Camera
camera = cv2.VideoCapture(0)
camera.set(10, 200)

False

In [81]:
startCapture = True #to capture the screen one time only
s='' #for the product Id string

# Whole Operation

In [82]:
while camera.isOpened():
    
    frame = getFrame(camera)
    
    # Run once background is captured
    if isBgCaptured == 1:
        thresh = getThresh(frame)
        
    # copies 1 channel BW image to all 3 RGB channels
        target = processTarget(thresh)
        action = "Hello!!"
        prediction, score = predict_rgb_image_vgg(target)
        if(prediction == "Peace" and score>=99.0):
            k = cv2.waitKey(10) 
            startTime = time.time()
            waitTime = time.time()-startTime
            while(waitTime<=5):
                waitTime = time.time()-startTime
                action = "There you go!"
                prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                k=cv2.waitKey(10)
            while(1):
                breakWhile = False
                waitTime = 0
                timeLeft = 0
                action = "Gesture & Select Digit"
                prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                k = cv2.waitKey(10)
                if(k==27):
                    break
                if(prediction == "L" and score>=99.0):
                    OkayWait()
                    startTime = time.time()
                    currentTime = time.time()-startTime
                    while(currentTime<=60):
                        currentTime = time.time()-startTime
                        timeLeft=60-currentTime
                        prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                        if(prediction == "Okay" and score==100.0):
                            action = "Selected 1"
                            s += "1"
                            nextWait()
                            totalProcess(camera)
                            break
                        k = cv2.waitKey(10)
                if(prediction == "Fist" and score==100.0):
                    OkayWait()
                    startTime=time.time()
                    currentTime=time.time()-startTime 
                    while(currentTime<=60):
                        currentTime=time.time()-startTime
                        timeLeft=60-currentTime
                        prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                        if(prediction == "Okay" and score==100.0):
                            action = "Selected 2"
                            s += "2"
                            nextWait()
                            totalProcess(camera)
                            break
                        k = cv2.waitKey(10)
                if(prediction == "Palm" and score==100.0):
                    OkayWait()
                    startTime = time.time()
                    currentTime = time.time()-startTime
                    while(currentTime<=60):
                        currentTime = time.time() - startTime
                        timeLeft = 60 - currentTime
                        prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                        if(prediction == "Okay" and score==100.0):
                            action = "Selected 3"
                            s += "3"
                            nextWait()
                            totalProcess(camera)
                            break
                        k = cv2.waitKey(10)
                if(prediction == "Peace" and score>=98.0):
                    OkayWait()
                    startTime=time.time()
                    currentTime=time.time()-startTime
                    while(currentTime<=60):
                        currentTime = time.time() - startTime
                        timeLeft = 60 - currentTime
                        prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                        if(prediction == "Okay" and score==100.0):
                            action = "Bye Bye"
                            print("Bye Bye")
                            totalProcess(camera)
                            breakWhile = True
                            break
                        k = cv2.waitKey(10)
                
                if(breakWhile==True):
                    break
                                
                         
        if(prediction == "Palm" and score>=98.0):
            k = cv2.waitKey(10) 
            startTime = time.time()
            waitTime = time.time()-startTime
            while(waitTime<=5):
                waitTime = time.time()-startTime
                action = "There you go!"
                prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                k=cv2.waitKey(10)
            while(1):
                breakWhile = False
                waitTime = 0
                timeLeft = 0
                action = "Gesture & Select Digit"
                prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                k = cv2.waitKey(10)
                if(k==27):
                    break
                if(prediction == "L" and score>=99.0):
                    OkayWait()
                    startTime = time.time()
                    currentTime = time.time()-startTime
                    while(currentTime<=60):
                        currentTime = time.time()-startTime
                        timeLeft = 60 - currentTime
                        prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                        if(prediction == "Okay" and score==100.0):
                            action = "Selected 4"
                            s += "4"
                            nextWait()
                            totalProcess(camera)
                            break
                        k = cv2.waitKey(10)
                if(prediction == "Fist" and score==100.0):
                    OkayWait()
                    startTime=time.time()
                    currentTime=time.time()-startTime 
                    while(currentTime<=60):
                        currentTime=time.time()-startTime
                        timeLeft=60-currentTime
                        prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                        if(prediction == "Okay" and score==100.0):
                            action = "Selected 5"
                            s += "5"
                            nextWait()
                            totalProcess(camera)
                            break
                        k = cv2.waitKey(10)
                if(prediction == "Peace" and score==100.0):
                    OkayWait()
                    startTime = time.time()
                    currentTime = time.time()-startTime
                    while(currentTime<=60):
                        currentTime = time.time() - startTime
                        timeLeft=60-currentTime
                        prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                        if(prediction == "Okay" and score==100.0):
                            action = "Selected 6"
                            s += "6"
                            nextWait()
                            totalProcess(camera)
                            break
                        k = cv2.waitKey(10)
                if(prediction == "Palm" and score>=98.0):
                    OkayWait()
                    startTime=time.time()
                    currentTime=time.time()-startTime
                    while(currentTime<=60):
                        currentTime = time.time() - startTime
                        timeLeft=60-currentTime
                        prediction, score = predict_rgb_image_vgg(totalProcess(camera))
                        if(prediction == "Okay" and score==100.0):
                            action = "Bye Bye"
#                             print("Bye Bye")
                            totalProcess(camera)
                            breakWhile = True
                            break
                        k = cv2.waitKey(10)
                
                if(breakWhile==True):
                    break
                    
        
        if(prediction=='L' and score==100.0):
            print(s)
            purchaseId=s
            totalProcess(camera)
            b = s.encode('utf-8')
            startTime=time.time()
            while(time.time()-startTime<=1):
                serialPort.write(b)
            s=''
    
     # Keyboard OP for emergency stop
    if(trigger==False):
        serialProcess()
        triggerCapture()
        
    k = cv2.waitKey(10)
    
    if k == 27:  # press ESC to exit all windows at any time
        break           
        
cv2.destroyAllWindows()
camera.release()

serialPort.close()

Hello from capture
Bye Bye
Bye Bye
15

